In [11]:
import numpy as np
from PIL import Image
from scipy.fftpack import dct, idct

In [2]:
def rgb_to_ycbcr(image):
    """
    Convert an RGB image to YCbCr and return separate Y, Cb, and Cr images.
    Parameters: a PIL Image object in RGB mode.
    Returns: Y, Cb, Cr: numpy arrays representing the Y, Cb, and Cr components.
    """
    img_array = np.array(image, dtype=float)
    
    # Separate the RGB channels
    R = img_array[:, :, 0]
    G = img_array[:, :, 1]
    B = img_array[:, :, 2]
    
    Y = 0.299 * R + 0.587 * G + 0.114 * B
    Cb = 128 - 0.168736 * R - 0.331264 * G + 0.5 * B
    Cr = 128 + 0.5 * R - 0.418688 * G - 0.081312 * B
    
    return Y, Cb, Cr

def save_component_image(component, filename):
    """
    Save a single Y, Cb, or Cr component as an image.
    
    Parameters:
    - component: a numpy array representing the Y, Cb, or Cr component.
    - filename: the name of the file to save the image as.
    """
    component_uint8 = np.uint8(component)
    component_pil = Image.fromarray(component_uint8)
    component_pil.save(filename)

def apply_420_subsampling(Cb, Cr):
    """
    Apply 4:2:0 subsampling to the Cb and Cr components.
    
    Parameters:
    - Cb: numpy array representing the Cb component.
    - Cr: numpy array representing the Cr component.
    
    Returns:
    - Cb_subsampled, Cr_subsampled: numpy arrays representing the subsampled Cb and Cr components.
    """
    # Perform 4:2:0 subsampling
    Cb_subsampled = Cb[::2, ::2]  # Take every second row and column
    Cr_subsampled = Cr[::2, ::2]  # Take every second row and column
    
    return Cb_subsampled, Cr_subsampled

def ycbcr_to_rgb(Y, Cb_subsampled, Cr_subsampled):
    """
    Convert YCbCr components back to RGB.
    
    Parameters:
    - Y: numpy array representing the Y component.
    - Cb_subsampled: numpy array representing the subsampled Cb component.
    - Cr_subsampled: numpy array representing the subsampled Cr component.
    
    Returns:
    - rgb_image: PIL Image object representing the reconstructed RGB image.
    """
    # Upsample Cb and Cr components to match Y size (repeat rows and columns)
    height, width = Y.shape
    Cb_upsampled = np.repeat(np.repeat(Cb_subsampled, 2, axis=0), 2, axis=1)
    Cr_upsampled = np.repeat(np.repeat(Cr_subsampled, 2, axis=0), 2, axis=1)
    
    # Perform inverse YCbCr to RGB conversion
    R = Y + 1.402 * (Cr_upsampled - 128)
    G = Y - 0.344136 * (Cb_upsampled - 128) - 0.714136 * (Cr_upsampled - 128)
    B = Y + 1.772 * (Cb_upsampled - 128)
    
    # Stack R, G, B channels and clip values to [0, 255]
    rgb_image = np.stack((R, G, B), axis=-1)
    rgb_image = np.clip(rgb_image, 0, 255)
    rgb_image = np.uint8(rgb_image)
    
    # Convert numpy array to PIL Image
    rgb_image_pil = Image.fromarray(rgb_image)
    
    return rgb_image_pil
# Example usage:
image_path = 'sample.bmp'
image = Image.open(image_path).convert('RGB')
Y, Cb, Cr = rgb_to_ycbcr(image)

# Save the Y, Cb, and Cr images
# save_component_image(Y, 'Y_component.bmp')
# save_component_image(Cb, 'Cb_component.bmp')
# save_component_image(Cr, 'Cr_component.bmp')

# To visualize the components
#Image.fromarray(np.uint8(Y)).show(title="Y Component")
#Image.fromarray(np.uint8(Cb)).show(title="Cb Component")
#Image.fromarray(np.uint8(Cr)).show(title="Cr Component")


Cb_subsampled, Cr_subsampled = apply_420_subsampling(Cb, Cr)
# Visualize the original and subsampled Cb and Cr components
#Image.fromarray(np.uint8(Cb)).show(title="Original Cb Component")
#Image.fromarray(np.uint8(Cb_subsampled)).show(title="Subsampled Cb Component")

#Image.fromarray(np.uint8(Cr)).show(title="Original Cr Component")
#Image.fromarray(np.uint8(Cr_subsampled)).show(title="Subsampled Cr Component")

# Convert YCbCr components back to RGB
#reconstructed_rgb_image = ycbcr_to_rgb(Y, Cb_subsampled, Cr_subsampled)

# Display the reconstructed RGB image
#reconstructed_rgb_image.show(title="Reconstructed RGB Image")


In [35]:

def block_process(channel, block_size, process_block):
    print(f"Processing blocks of shape: {channel.shape}")
    h, w = channel.shape[:2]
    blocks = (channel.reshape(h // block_size, block_size, -1, block_size)
                      .swapaxes(1, 2)
                      .reshape(-1, block_size, block_size))
    processed_blocks = np.array([process_block(block) for block in blocks])
    return (processed_blocks.reshape(h // block_size, w // block_size, block_size, block_size)
                            .swapaxes(1, 2)
                            .reshape(h, w))
def dct2(block):
    return dct(dct(block.T, norm='ortho').T, norm='ortho')


def dct_2d(block):
    N = block.shape[0]
    dct_matrix = np.zeros((N, N))

    def alpha(u):
        return np.sqrt(1/2) if u == 0 else 1

    for u in range(N):
        for v in range(N):
            sum_value = 0.0
            for x in range(N):
                for y in range(N):
                    sum_value += block[x, y] * np.cos((2 * x + 1) * u * np.pi / (2 * N)) * np.cos((2 * y + 1) * v * np.pi / (2 * N))
            dct_matrix[u, v] = 0.25 * alpha(u) * alpha(v) * sum_value

    return dct_matrix

In [36]:
y_dct = block_process(Y, 8, dct2)
cb_dct = block_process(Cb_subsampled, 8, dct2)
cr_dct = block_process(Cr_subsampled, 8, dct2)


Processing blocks of shape: (1280, 1920)
Processing blocks of shape: (640, 960)
Processing blocks of shape: (640, 960)


In [37]:
channel = Y
block_size = 8
h, w = channel.shape[:2]
blocks = (channel.reshape(h // block_size, block_size, -1, block_size)
                      .swapaxes(1, 2)
                      .reshape(-1, block_size, block_size))
print(blocks[0])
print(dct2(blocks[0]))
print(dct_2d(blocks[0]))

[[144.008 150.008 145.008 148.008 138.008 145.008 137.008 136.008]
 [150.008 144.008 143.008 155.008 147.008 138.008 136.008 150.008]
 [142.008 138.008 140.008 140.008 147.008 147.008 149.008 146.008]
 [146.008 147.008 150.008 137.008 144.008 140.008 142.008 131.008]
 [147.008 141.008 150.008 151.008 153.008 133.008 139.236 147.236]
 [147.008 139.008 140.008 145.008 155.008 151.008 148.236 145.236]
 [156.008 154.008 144.008 140.008 142.236 151.236 141.236 127.236]
 [142.008 146.008 146.008 154.008 136.236 134.236 138.236 159.236]]
[[ 1.15453100e+03  1.17733301e+01 -3.21232013e+00 -2.08314387e-01
   7.62500000e+00  2.99110354e-02  1.12895341e-01 -2.34241516e-01]
 [-4.14751481e+00  3.14828975e-01 -3.54427882e+00 -8.28988716e-03
  -9.55155077e-02  1.85248738e-01 -4.59181983e-02 -5.77475168e-01]
 [ 3.76727542e-02  4.38833857e+00  5.03918030e+00 -7.49843690e+00
   3.94290254e-01  3.09611952e-02  6.07246825e-01 -1.66151644e+01]
 [ 3.97960028e+00  4.80279033e+00 -6.84366499e+00  8.88136234e+0